In [18]:
from torchvision import models, transforms
import torch.nn as nn
import torch
import os
from skimage import io
from pathlib import Path
import dropbox
import shutil

In [37]:
def model_predict(model_name):
    model_eval = models.resnet101(pretrained=True)
    class_output = {
    'binary': {'Ghost': 0, 'Animal': 1, 'Unknown': 2},
    'class': {'Ghost': 0, 'Aves': 1, 'Mammalia': 2, 'Unknown': 3},
    'species':{
                'Ghost': 0,
                'ArremonAurantiirostris': 1,
                'Aves (Class)': 2,
                'BosTaurus': 3,
                'CaluromysPhilander': 4,
                'CerdocyonThous': 5,
                'CuniculusPaca': 6,
                'Dasyprocta (Genus)': 7,
                'DasypusNovemcinctus': 8,
                'DidelphisAurita': 9,
                'EiraBarbara': 10,
                'Equus (Genus)': 11,
                'Leopardus (Genus)': 12,
                'LeptotilaPallida': 13,
                'Mammalia (Class)': 14,
                'MazamaAmericana': 15,
                'Metachirus (Genus)': 16,
                'Momota (Genus)': 17,
                'Nasua (Genus)': 18,
                'PecariTajacu': 19,
                'ProcyonCancrivorus': 20,
                'Rodentia (Order)': 21,
                'Sciurus (Genus)': 22,
                'SusScrofa': 23,
                'TamanduaTetradactyla': 24,
                'TinamusMajor': 25,
                'Unknown': 26}
}

    model_map = class_output[model_name]
    in_features =len(model_map)
    reverse_model_map = {v: k for k, v in model_map.items()}
    
    using_gpu = torch.cuda.is_available()
    if using_gpu:
        print('Image Prediction Starts. Using GPU!')
    else:
        print('Image Prediction Starts. Using CPU!')
    device = torch.device("cuda:0" if using_gpu else "cpu")
    
    num_ftrs = model_eval.fc.in_features
    model_eval.fc = nn.Linear(num_ftrs, in_features)

    model_eval = model_eval.to(device)    
    
    model_full_name = sorted([i for i in os.listdir('./models/') if model_name.lower() in i.split('_')], reverse=True)[0]
    
    try:
        model_eval.load_state_dict(torch.load(f'./models/{model_full_name}',  map_location=device))
        print(f'Loading {model_full_name} pre-trained model')
    except Exception as e:
        print(f'cannot load model! {e}')
        
    regulated_size = 300, 450
    default_val_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize(size=regulated_size),
        transforms.ToTensor(),
    ])
    model_eval.eval()
    softmax = torch.nn.Softmax(dim=1) 
    
    # Dropbox API
    access_token = "IdSksADBFacAAAAAAAAAAcTAT5d77_SGkwjhmQcxC2BHo8ak0B1j_lGKiZqfH3cy"
    dbx = dropbox.Dropbox(access_token)
    
    img_dropbox = '/MIDS/Input_Data/'
    img_dropbox_to = '/MIDS/Output_Data/'
    
    img_local = Path('cache/')
    img_local.mkdir(exist_ok=True)
    folder = dbx.files_list_folder(img_dropbox)
    for file in folder.entries:
        if file.name.lower().endswith('jpg'):
            img_local_path = img_local / file.name
            dbx.files_download_to_file(download_path=img_local_path, path=file.path_display)
            tmp_img = io.imread(img_local_path)
            tmp_img = default_val_transform(tmp_img)

            with torch.no_grad():
                inputs = tmp_img.to(device)
                inputs = inputs.reshape(1, *inputs.shape)
                outputs = model_eval(inputs)

                prob = softmax(outputs)
                pred_prob, pred_id = torch.max(prob, 1)
                pred_id = pred_id.tolist()

                pred_prob = pred_prob.tolist()[0]
                pred_str = reverse_model_map[pred_id[0]]
            print(f'Image Name: {file.name}; Predicted Class: {pred_str}; Predicted Probability:{pred_prob * 100:.2f}%')
            dbx.files_copy_v2(from_path=file.path_display, to_path=f'{img_dropbox_to}{model_name}/{pred_str}/{file.name}', autorename=True)  
            
    shutil.rmtree(img_local)
    print('Image Prediction Finished!')

In [38]:
model_predict('class')

Image Prediction Starts. Using CPU!
Loading resnet101_class_20210416.pth pre-trained model
Image Name: 000045_brazilAMLD_SSAMLD2_2_Aves_01_2019_03_24_12_14_02.jpg; Predicted Class: Aves; Predicted Probability:98.34%
Image Name: 000036_brazilAMLD_SSAMLD2_2_Aves_01_2019_02_07_18_14_47.jpg; Predicted Class: Unknown; Predicted Probability:92.35%
Image Prediction Finished!


In [2]:
model_name = 'species'

In [3]:
model_eval = models.resnet101(pretrained=True)

In [4]:
class_output = {
    'binary': {'Ghost': 0, 'Animal': 1, 'Unknown': 2},
    'class': {'Ghost': 0, 'Aves': 1, 'Mammalia': 2, 'Unknown': 3},
    'species':{
                'Ghost': 0,
                'ArremonAurantiirostris': 1,
                'Aves (Class)': 2,
                'BosTaurus': 3,
                'CaluromysPhilander': 4,
                'CerdocyonThous': 5,
                'CuniculusPaca': 6,
                'Dasyprocta (Genus)': 7,
                'DasypusNovemcinctus': 8,
                'DidelphisAurita': 9,
                'EiraBarbara': 10,
                'Equus (Genus)': 11,
                'Leopardus (Genus)': 12,
                'LeptotilaPallida': 13,
                'Mammalia (Class)': 14,
                'MazamaAmericana': 15,
                'Metachirus (Genus)': 16,
                'Momota (Genus)': 17,
                'Nasua (Genus)': 18,
                'PecariTajacu': 19,
                'ProcyonCancrivorus': 20,
                'Rodentia (Order)': 21,
                'Sciurus (Genus)': 22,
                'SusScrofa': 23,
                'TamanduaTetradactyla': 24,
                'TinamusMajor': 25,
                'Unknown': 26}
}

In [5]:
model_map = class_output[model_name]
in_features =len(model_map)
reverse_model_map = {v: k for k, v in model_map.items()}

In [6]:
using_gpu = torch.cuda.is_available()
if using_gpu:
    print('Using GPU!')
else:
    print('Using CPU!')
device = torch.device("cuda:0" if using_gpu else "cpu")

Using CPU!


In [7]:
num_ftrs = model_eval.fc.in_features
model_eval.fc = nn.Linear(num_ftrs, in_features)

model_eval = model_eval.to(device)

In [8]:
model_full_name = sorted([i for i in os.listdir('./models/') if model_name.lower() in i.split('_')], reverse=True)[0]

In [9]:
model_full_name

'resnet101_species_20210419.pth'

In [10]:
try:
    model_eval.load_state_dict(torch.load(f'./models/{model_full_name}',  map_location=device))
    print(f'Loading {model_full_name} pre-trained model')
except Exception as e:
    print(f'cannot load model! {e}')

Loading resnet101_species_20210419.pth pre-trained model


In [11]:
regulated_size = 300, 450
default_val_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(size=regulated_size),
    transforms.ToTensor(),
])
model_eval.eval()
softmax = torch.nn.Softmax(dim=1)

In [12]:
# Dropbox API
access_token = "IdSksADBFacAAAAAAAAAAcTAT5d77_SGkwjhmQcxC2BHo8ak0B1j_lGKiZqfH3cy"
dbx = dropbox.Dropbox(access_token)

In [31]:
img_dropbox = '/MIDS/Input_Data/'
img_dropbox_to = '/MIDS/Output_Data/'

In [34]:
img_local = Path('cache/')
img_local.mkdir(exist_ok=True)
folder = dbx.files_list_folder(img_dropbox)
for file in folder.entries:
    if file.name.lower().endswith('jpg'):
        img_local_path = img_local / file.name
        dbx.files_download_to_file(download_path=img_local_path, path=file.path_display)
        tmp_img = io.imread(img_local_path)
        tmp_img = default_val_transform(tmp_img)
        
        with torch.no_grad():
            inputs = tmp_img.to(device)
            inputs = inputs.reshape(1, *inputs.shape)
            outputs = model_eval(inputs)

            prob = softmax(outputs)
            pred_prob, pred_id = torch.max(prob, 1)
            pred_id = pred_id.tolist()

            pred_prob = pred_prob.tolist()[0]
            pred_str = reverse_model_map[pred_id[0]]
        print(f'Image Name: {file.name}; Predicted Class: {pred_str}; Predicted Probability:{pred_prob * 100:.2f}%')
        dbx.files_copy_v2(from_path=file.path_display, to_path=f'{img_dropbox_to}{model_name}/{pred_str}/{file.name}', autorename=True)

Image Name: 000045_brazilAMLD_SSAMLD2_2_Aves_01_2019_03_24_12_14_02.jpg; Predicted Class: Aves (Class); Predicted Probability:99.88%
Image Name: 000036_brazilAMLD_SSAMLD2_2_Aves_01_2019_02_07_18_14_47.jpg; Predicted Class: Unknown; Predicted Probability:66.94%


In [22]:
shutil.rmtree(img_local)